In [31]:
# Import the required libraries
import csv
import os
import time
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup
from selenium.webdriver.common.proxy import Proxy, ProxyType
from urllib.parse import urlparse
import re
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [22]:
# Setup the driver
url = "https://www.woolworths.com.au" 
firefox_option = webdriver.FirefoxOptions()
firefox_option.add_argument("--headless")
driver = webdriver.Firefox(options=firefox_option)

In [23]:
driver.get(url + "/shop/browse/bakery")

In [24]:
print(driver.current_url)

https://www.woolworths.com.au/shop/browse/bakery


In [25]:
# Create a BeautifulSoup object
time.sleep(5)
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Find all the product tiles
product_tiles = soup.find_all('div', class_='product-grid-v2--tile')

for tile in product_tiles:
    # Extract the product name
    product_name_elem = tile.find('a', class_='product-title-link')
    if not product_name_elem:
        continue
    product_name = product_name_elem.text.strip()
    
    # Extract the price
    currency = tile.find('div', class_="primary").text.strip() if tile.find('div', class_="primary") else "N/A"
    unit_price = tile.find('span', class_='price-per-cup').text.strip() if tile.find('span', class_='price-per-cup') else "N/A"
    
    # Extract the link
    link_elem = tile.find('a', {'aria-label': True})
    link = link_elem['href'] if link_elem else "N/A"
    
    # Print the information
    print("Product Name:", product_name)
    print("Currency:", currency)
    print("Unit Price:", unit_price)
    print("Link:", link)
    print()

Product Name: Woolworths White Wrap 8 Pack
Currency: $2.50
Unit Price: $0.69 / 100G
Link: /shop/productdetails/634918/woolworths-white-wrap

Product Name: Mission Wraps Original 8 Pack
Currency: $5.50
Unit Price: $0.97 / 100G
Link: /shop/productdetails/320194/mission-wraps-original

Product Name: Woolworths Wholegrain Wrap 8pk 360g
Currency: $2.50
Unit Price: $0.69 / 100G
Link: /shop/productdetails/634919/woolworths-wholegrain-wrap-8pk

Product Name: Mission Wraps Wholegrain 8 Pack 567g
Currency: $5.50
Unit Price: $0.97 / 100G
Link: /shop/productdetails/384286/mission-wraps-wholegrain

Product Name: Woolworths Stonebaked Turkish Rolls 4 Pack
Currency: $2.80
Unit Price: $0.70 / 100G
Link: /shop/productdetails/85969/woolworths-stonebaked-turkish-rolls

Product Name: Woolworths 35hr Sourdough Loaf White Each
Currency: $6.50
Unit Price: $6.50 / 1EA
Link: /shop/productdetails/363952/woolworths-35hr-sourdough-loaf-white-loaf

Product Name: Mission Wraps Spinach & Herb 8 Pack
Currency: $5.50


## Complete App

In [2]:
# Import the required libraries
import csv
import os
import time
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup
from selenium.webdriver.common.proxy import Proxy, ProxyType
from urllib.parse import urlparse
import re
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [10]:
wait_time = 6

firefox_option = webdriver.FirefoxOptions()
# firefox_option.add_argument("--headless")
driver = webdriver.Firefox(options=firefox_option)
url = "https://www.woolworths.com.au"
print("Here we go...")
driver.get(url + "/browse")
# "fruit-veg", "bakery", "lunch-box", "poultry-meat-seafood", "deli-chilled-meals", "dairy-eggs-fridge", "pantry"
categories = ["health-wellness/health-foods", "freezer","drinks"]
for category in categories:
    # Get the link to the category page
    category_link = url + "/shop/browse/" + category
    # Follow the link to the category page
    driver.get(category_link)
    time.sleep(wait_time)
    # Wait until the page is fully loaded
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a.product-title-link[aria-label]')))
    # Parse the page content
    soup = BeautifulSoup(driver.page_source, "html.parser") 
    while True:
        # Create a new csv file for each category
        filename = category + ".csv"
        if category == "health-wellness/health-foods":
            filepath = "C:/Users/nplon/Documents/Woolies health-wellness health-foods"
        else:
            filepath = "C:/Users/nplon/Documents/Woolies " + filename
        if os.path.exists(filepath):
            os.remove(filepath)
        with open(filepath, "a", newline="") as f:
            writer = csv.writer(f)
            
            # Find all the product tiles
            product_tiles = soup.find_all('div', class_='product-grid-v2--tile')

            for tile in product_tiles:
                # Extract the product name
                product_name_elem = tile.find('a', class_='product-title-link')
                if not product_name_elem:
                    continue
                product_name = product_name_elem.text.strip()
                
                # Extract the price
                currency = tile.find('div', class_="primary").text.strip() if tile.find('div', class_="primary") else "N/A"
                unit_price = tile.find('span', class_='price-per-cup').text.strip() if tile.find('span', class_='price-per-cup') else "N/A"
                
                # Extract the link
                link_elem = tile.find('a', {'aria-label': True})
                link = link_elem['href'] if link_elem and 'href' in link_elem.attrs else "N/A"
                

                # Extract the ID
                match = re.search(r"/(\d+)/", link)
                if match:
                    product_id = match.group(1)
                
                # Print the information
                print("Product Name:", product_name)
                print("Currency:", currency)
                print("Unit Price:", unit_price)
                print("Link:", link)
                print("Id:", product_id)
                print()
                        
            # Get the number of pages
            try:
                pagination = soup.find("div", class_="paging-section")
            except:
                break

            last = soup.find("div", class_="paging-section").find_all("a")[-2].text
            print(re.findall(r'\d+', last)[0])

            pages = pagination.find_all("a")
            last_page = pages[-2].text
            last_page = re.findall(r'\d+', last)[0]
            total_pages = re.findall(r'\d+', last)[0]
            print(total_pages)
            # Pantry: start at page 46 instead of 2
            if category == "pantry":
                start = 56
            else:
                start = 2

            for page in range(start, int(last_page) + 1):
                # Get the link to the next page
                next_page_link = f"{category_link}?pageNumber={page}"
                # Navigate to the next page
                driver.get(next_page_link)

                time.sleep(wait_time)
                # Parse the page content
                soup = BeautifulSoup(driver.page_source, "html.parser")

                # Find all the product tiles
                product_tiles = soup.find_all('div', class_='product-grid-v2--tile')

                for tile in product_tiles:
                    # Extract the product name
                    product_name_elem = tile.find('a', class_='product-title-link')
                    if not product_name_elem:
                        continue
                    product_name = product_name_elem.text.strip()
                    
                    # Extract the price
                    currency = tile.find('div', class_="primary").text.strip() if tile.find('div', class_="primary") else "N/A"
                    unit_price = tile.find('span', class_='price-per-cup').text.strip() if tile.find('span', class_='price-per-cup') else "N/A"
                    
                    # Extract the link
                    link_elem = tile.find('a', {'aria-label': True})
                    link = link_elem['href'] if link_elem and 'href' in link_elem.attrs else "N/A"

                    # Extract the ID
                    match = re.search(r"/(\d+)/", link)
                    product_id = match.group(1) if link != "N/A" else "N/A"

                    # RESCRAPE if no link -> no ID
                    if link == "N/A":
                        driver.get(next_page_link)
                        time.sleep(wait_time + 3)
                        # Parse the page content
                        soup = BeautifulSoup(driver.page_source, "html.parser")
                        # Find all the product tiles
                        product_tiles = soup.find_all('div', class_='product-grid-v2--tile')

                        for tile in product_tiles:
                            # Extract the product name
                            product_name_elem = tile.find('a', class_='product-title-link')
                            if not product_name_elem:
                                continue
                            product_name = product_name_elem.text.strip()
                            
                            # Extract the price
                            currency = tile.find('div', class_="primary").text.strip() if tile.find('div', class_="primary") else "N/A"
                            unit_price = tile.find('span', class_='price-per-cup').text.strip() if tile.find('span', class_='price-per-cup') else "N/A"
                            
                            # Extract the link
                            link_elem = tile.find('a', {'aria-label': True})
                            link = link_elem['href'] if link_elem and 'href' in link_elem.attrs else "N/A"

                            # Extract the ID
                            match = re.search(r"/(\d+)/", link)
                            product_id = match.group(1) if link != "N/A" else "N/A"
                        break
                    
                    # Print the information
                    print("Product Name:", product_name)
                    print("Currency:", currency)
                    print("Unit Price:", unit_price)
                    print("Link:", link)
                    print("Id:", product_id)
                    print()
                    writer.writerow([product_id, product_name, currency, unit_price, link])
            # time.sleep(3)
            if int(page) == int(last_page):
                break
print("Finished")

Here we go...
Product Name: Chief Nutrition Beauty Food Collagen Cookie Choc Chic 30g
Currency: $4.00
Unit Price: $13.33 / 100G
Link: /shop/productdetails/202633/chief-nutrition-beauty-food-collagen-cookie-choc-chic
Id: 202633

Product Name: Chief Nutrition Chief Collagen Bar Hazelnut Brownie 45g
Currency: $5.00
Unit Price: $11.11 / 100G
Link: /shop/productdetails/203076/chief-nutrition-chief-collagen-bar-hazelnut-brownie
Id: 203076

Product Name: Chief Nutrition Beauty Food Collagen Cookie Peanut Nutter 30g
Currency: $4.00
Unit Price: $13.33 / 100G
Link: /shop/productdetails/202668/chief-nutrition-beauty-food-collagen-cookie-peanut-nutter
Id: 202668

Product Name: Chief Nutrition Chief Collagen Bar Cashew Shortbread 45g
Currency: $5.00
Unit Price: $11.11 / 100G
Link: /shop/productdetails/203071/chief-nutrition-chief-collagen-bar-cashew-shortbread
Id: 203071

Product Name: Woolworths Sesame Seeds 100g
Currency: $1.10
Unit Price: $11.00 / 1KG
Link: /shop/productdetails/501762/woolworths